# Mesh generation for tutorial 06, case 3b

This file generates the mesh which is used in the following examples:
* 3b_advection_diffusion_reaction_neumann_control

The test case is from section 5.3 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import os
import meshio
import pygmsh
from dolfinx import MPI
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters and related quantities

In [ ]:
lcar = 0.05

### Create mesh

In [ ]:
class GenerateRectangleLines(object):
    def __init__(self, geom):
        self.geom = geom
        self.points = dict()
        self.lines = dict()

    def add_point(self, x, y):
        key = (x, y)
        try:
            return self.points[key]
        except KeyError:
            p = self.geom.add_point([x, y, 0.0], lcar)
            self.points[key] = p
            return p

    def add_line(self, p0, p1):
        try:
            return self.lines[(p0, p1)]
        except KeyError:
            l01 = self.geom.add_line(p0, p1)
            self.lines[(p0, p1)] = l01
            self.lines[(p1, p0)] = -l01
            return l01

    def __call__(self, x_min, x_max, y_min, y_max):
        p0 = self.add_point(x_min, y_min)
        p1 = self.add_point(x_max, y_min)
        if isinstance(y_max, list):
            p2 = [self.add_point(x_max, y) for y in y_max]
            p3 = self.add_point(x_min, y_max[-1])
        else:
            p2 = self.add_point(x_max, y_max)
            p3 = self.add_point(x_min, y_max)
        l0 = self.add_line(p0, p1)
        if isinstance(y_max, list):
            p1_p2 = [p1] + p2
            l1 = [self.add_line(p1_p2[i], p1_p2[i + 1]) for i in range(len(p2))]
            l2 = self.add_line(p2[-1], p3)
        else:
            l1 = self.add_line(p1, p2)
            l2 = self.add_line(p2, p3)
        l3 = self.add_line(p3, p0)
        return (l0, l1, l2, l3)


geom = pygmsh.built_in.Geometry()
generate_rectangle_lines = GenerateRectangleLines(geom)
[l0, l1, l2, l3] = generate_rectangle_lines(0.0, 1.0, 0.0, [0.3, 0.7, 1.0])
[l4, l5, l6, _] = generate_rectangle_lines(1.0, 3.0, 0.0, 0.3)
[_, l7, l8, _] = generate_rectangle_lines(1.0, 3.0, 0.3, 0.7)
[_, l9, l10, _] = generate_rectangle_lines(1.0, 3.0, 0.7, 1.0)
geom.add_physical([l0, l2, l3], label=1)
geom.add_physical([l4, l10], label=2)
geom.add_physical([l5, l7, l9], label=3)
line_loop_rectangle_left = geom.add_line_loop([l0, l1[0], l1[1], l1[2], l2, l3])
line_loop_rectangle_right_bottom = geom.add_line_loop([l4, l5, l6, -l1[0]])
line_loop_rectangle_right_middle = geom.add_line_loop([-l6, l7, l8, -l1[1]])
line_loop_rectangle_right_top = geom.add_line_loop([-l8, l9, l10, -l1[2]])
rectangle_left = geom.add_plane_surface(line_loop_rectangle_left)
rectangle_right_bottom = geom.add_plane_surface(line_loop_rectangle_right_bottom)
rectangle_right_middle = geom.add_plane_surface(line_loop_rectangle_right_middle)
rectangle_right_top = geom.add_plane_surface(line_loop_rectangle_right_top)
geom.add_physical(rectangle_left, label=11)
geom.add_physical([rectangle_right_bottom, rectangle_right_top], label=13)
geom.add_physical(rectangle_right_middle, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Save temporary mesh and subdomains (pygmsh format)

In [ ]:
meshio.write("graetz_2_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"triangle": pygmsh_mesh.cells_dict["triangle"]},
    cell_data={"subdomains": [pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10]}
))

### Save temporary boundaries (pygmsh format)

In [ ]:
meshio.write("graetz_2_boundaries_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"line": pygmsh_mesh.cells_dict["line"]},
    cell_data={"boundaries": [pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]]}
))

### Read back in mesh (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "graetz_2_tmp.xdmf", "r") as infile:
    mesh = infile.read_mesh(name="Grid")
    mesh.name = "mesh"

In [ ]:
plot(mesh)

### Read back in subdomains and boundaries (dolfinx format)

In [ ]:
mesh.create_connectivity_all()
with XDMFFile(MPI.comm_world, "graetz_2_tmp.xdmf", "r") as infile:
    subdomains = infile.read_meshtags(mesh, name="Grid")
    subdomains.name = "subdomains"
with XDMFFile(MPI.comm_world, "graetz_2_boundaries_tmp.xdmf", "r") as infile:
    boundaries = infile.read_meshtags(mesh, name="Grid")
    boundaries.name = "boundaries"

### Save final mesh, subdomains and boundaries (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "graetz_2.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    file.write_meshtags(boundaries)

In [ ]:
os.remove("graetz_2_tmp.xdmf")
os.remove("graetz_2_tmp.h5")
os.remove("graetz_2_boundaries_tmp.xdmf")
os.remove("graetz_2_boundaries_tmp.h5")